In [1]:
import os, glob, random
import pandas as pd
import SimpleITK as sitk
import numpy as np

def get_image_path_by_id(patient_id, image_dir):
    image_order  = patient_id
    file_name = [os.path.relpath(os.path.join(image_dir, x)) \
                    for x in os.listdir(image_dir) \
                    if os.path.isfile(os.path.join(image_dir, x)) and patient_id in x][0] 
    return file_name

def get_msk_prd_img_cube_by_id(pid, manual_seg_dir, auto_seg_dir, img_dir ):
    
    auto_seg_path = get_image_path_by_id(pid, auto_seg_dir)
    manual_seg_path = get_image_path_by_id(pid, manual_seg_dir)
    img_path = get_image_path_by_id(pid, img_dir)

    manual_seg_sitk = sitk.ReadImage(manual_seg_path)
    mskCube  = sitk.GetArrayFromImage(manual_seg_sitk)
    auto_seg_sitk = sitk.ReadImage(auto_seg_path)
    prdCube = sitk.GetArrayFromImage(auto_seg_sitk)
    img_sitk = sitk.ReadImage(img_path)
    imgCube  = sitk.GetArrayFromImage(img_sitk)  
    
    return mskCube, prdCube, imgCube
    
def get_cube_by_class(cube_array):
    
    cube1 = np.copy(cube_array)
    cube1[cube1!=1]=0    
    cube1[cube1!=0]=1

    cube2 = np.copy(cube_array)
    cube2[cube2!=2]=0
    cube2[cube2!=0]=1
    
    cube3 = np.copy(cube_array)
    cube3[cube3!=3]=0
    cube3[cube3!=0]=1
    
    cube4 = np.copy(cube_array)
    cube4[cube4!=4]=0
    cube4[cube4!=0]=1    
    
    return cube1, cube2, cube3, cube4

def get_ratio_of_dixon_image_pair(mskCube_fat, imgCube_fat, mskCube_water, imgCube_water):
    
    fat_ratio = np.sum(np.multiply(mskCube_fat, imgCube_fat)) / \
                    np.sum((np.multiply(mskCube_fat, imgCube_fat) + np.multiply(mskCube_water, imgCube_water)))
    
    return fat_ratio
    

    

    

In [2]:
df_init = pd.DataFrame()


manual_seg_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/label_bedrest/'
auto_seg_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/seg_bedrest/'
img_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/img_bedrest/'
csv_write_path= 'csv_and_figure/MRI_Bedrest_fat_ratio_BDC_24.csv'

pid_list = []

for idx, d in sorted(enumerate(os.listdir(manual_seg_dir))):
    if '.nii.gz' in d :
#         print(d,'processing')

        indices = [i for i, c in enumerate(d) if c == '_']
        pid = d[:indices[0]]
        if pid not in pid_list:
            pid_list.append(pid)
            print(pid)
            pid_fat = pid + '_BDCFat'
            pid_water = pid + '_BDCWater'
            
            mskCube_fat, prdCube_fat, imgCube_fat = \
                get_msk_prd_img_cube_by_id(pid_fat, manual_seg_dir, auto_seg_dir, img_dir )
            
            mskCube_water, prdCube_water, imgCube_water = \
                get_msk_prd_img_cube_by_id(pid_water, manual_seg_dir, auto_seg_dir, img_dir)
            
            mskCube_fat1, mskCube_fat2, mskCube_fat3, mskCube_fat4 = get_cube_by_class(mskCube_fat)
            mskCube_water1, mskCube_water2, mskCube_water3, mskCube_water4 = get_cube_by_class(mskCube_water)            
            prdCube_fat1, prdCube_fat2, prdCube_fat3, prdCube_fat4 = get_cube_by_class(prdCube_fat)  
            prdCube_water1, prdCube_water2, prdCube_water3, prdCube_water4 = get_cube_by_class(prdCube_water)  
            
            
            
            ratio1_gd = get_ratio_of_dixon_image_pair(mskCube_fat1, imgCube_fat, mskCube_water1, imgCube_water)
            ratio2_gd = get_ratio_of_dixon_image_pair(mskCube_fat2, imgCube_fat, mskCube_water2, imgCube_water)
            ratio3_gd = get_ratio_of_dixon_image_pair(mskCube_fat3, imgCube_fat, mskCube_water3, imgCube_water)
            ratio4_gd= get_ratio_of_dixon_image_pair(mskCube_fat4, imgCube_fat, mskCube_water4, imgCube_water)   
            
            ratio1_ai = get_ratio_of_dixon_image_pair(prdCube_fat1, imgCube_fat, prdCube_fat1, imgCube_water)
            ratio2_ai = get_ratio_of_dixon_image_pair(prdCube_fat2, imgCube_fat, prdCube_fat2, imgCube_water)
            ratio3_ai = get_ratio_of_dixon_image_pair(prdCube_fat3, imgCube_fat, prdCube_fat3, imgCube_water)
            ratio4_ai = get_ratio_of_dixon_image_pair(prdCube_fat4, imgCube_fat, prdCube_fat4, imgCube_water) 
            
            df = pd.DataFrame({'MRI_id': pid + '_BDC',
                               
                        'ratio1_gd':round(ratio1_gd,5), 'ratio1_ai':round(ratio1_ai,5),
                        'ratio2_gd':round(ratio2_gd,5), 'ratio2_ai':round(ratio2_ai,5),
                        'ratio3_gd':round(ratio3_gd,5), 'ratio3_ai':round(ratio3_ai,5),
                        'ratio4_gd':round(ratio4_gd,5), 'ratio4_ai':round(ratio4_ai,5)
                                                 
                            },index=[0])
            df_init = df_init._append(df)
            df_init.to_csv(csv_write_path )
            print(idx,'th',pid,' done \n\n')
            
            print('\n\n\n')


ID4D
0 th ID4D  done 






ID18T
1 th ID18T  done 






ID20V
2 th ID20V  done 






ID13N
3 th ID13N  done 






ID24Z
5 th ID24Z  done 






ID17S
6 th ID17S  done 






ID7G
7 th ID7G  done 






ID19U
10 th ID19U  done 






ID12M
11 th ID12M  done 






ID11L
12 th ID11L  done 






ID9J
13 th ID9J  done 






ID22X
14 th ID22X  done 






ID8H
15 th ID8H  done 






ID16R
18 th ID16R  done 






ID21W
19 th ID21W  done 






ID15Q
22 th ID15Q  done 






ID23Y
28 th ID23Y  done 






ID1A
36 th ID1A  done 






ID2B
41 th ID2B  done 






ID10K
45 th ID10K  done 






ID5E
52 th ID5E  done 






ID14P
53 th ID14P  done 






ID3C
66 th ID3C  done 






ID6F
87 th ID6F  done 








In [3]:
df_init = pd.DataFrame()


manual_seg_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/label_bedrest/'
auto_seg_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/seg_bedrest/'
img_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/img_bedrest/'
csv_write_path= 'csv_and_figure/MRI_Bedrest_fat_ratio_R0_24.csv'

pid_list = []

for idx, d in sorted(enumerate(os.listdir(manual_seg_dir))):
    if '.nii.gz' in d :
#         print(d,'processing')

        indices = [i for i, c in enumerate(d) if c == '_']
        pid = d[:indices[0]]
        if pid not in pid_list:
            pid_list.append(pid)
            print(pid)
            pid_fat = pid + '_R0Fat'
            pid_water = pid + '_R0Water'
            
            mskCube_fat, prdCube_fat, imgCube_fat = \
                get_msk_prd_img_cube_by_id(pid_fat, manual_seg_dir, auto_seg_dir, img_dir )
            
            mskCube_water, prdCube_water, imgCube_water = \
                get_msk_prd_img_cube_by_id(pid_water, manual_seg_dir, auto_seg_dir, img_dir)
            
            mskCube_fat1, mskCube_fat2, mskCube_fat3, mskCube_fat4 = get_cube_by_class(mskCube_fat)
            mskCube_water1, mskCube_water2, mskCube_water3, mskCube_water4 = get_cube_by_class(mskCube_water)            
            prdCube_fat1, prdCube_fat2, prdCube_fat3, prdCube_fat4 = get_cube_by_class(prdCube_fat)  
            prdCube_water1, prdCube_water2, prdCube_water3, prdCube_water4 = get_cube_by_class(prdCube_water)  
            
            
            
            ratio1_gd = get_ratio_of_dixon_image_pair(mskCube_fat1, imgCube_fat, mskCube_water1, imgCube_water)
            ratio2_gd = get_ratio_of_dixon_image_pair(mskCube_fat2, imgCube_fat, mskCube_water2, imgCube_water)
            ratio3_gd = get_ratio_of_dixon_image_pair(mskCube_fat3, imgCube_fat, mskCube_water3, imgCube_water)
            ratio4_gd= get_ratio_of_dixon_image_pair(mskCube_fat4, imgCube_fat, mskCube_water4, imgCube_water)   
            
            ratio1_ai = get_ratio_of_dixon_image_pair(prdCube_fat1, imgCube_fat, prdCube_fat1, imgCube_water)
            ratio2_ai = get_ratio_of_dixon_image_pair(prdCube_fat2, imgCube_fat, prdCube_fat2, imgCube_water)
            ratio3_ai = get_ratio_of_dixon_image_pair(prdCube_fat3, imgCube_fat, prdCube_fat3, imgCube_water)
            ratio4_ai = get_ratio_of_dixon_image_pair(prdCube_fat4, imgCube_fat, prdCube_fat4, imgCube_water) 
            
            df = pd.DataFrame({'MRI_id': pid + '_R0',
                               
                        'ratio1_gd':round(ratio1_gd,5), 'ratio1_ai':round(ratio1_ai,5),
                        'ratio2_gd':round(ratio2_gd,5), 'ratio2_ai':round(ratio2_ai,5),
                        'ratio3_gd':round(ratio3_gd,5), 'ratio3_ai':round(ratio3_ai,5),
                        'ratio4_gd':round(ratio4_gd,5), 'ratio4_ai':round(ratio4_ai,5)
                                                 
                            },index=[0])
            df_init = df_init._append(df)
            df_init.to_csv(csv_write_path )
            print(idx,'th',pid,' done \n\n')
            
            print('\n\n\n')


ID4D
0 th ID4D  done 






ID18T
1 th ID18T  done 






ID20V
2 th ID20V  done 






ID13N
3 th ID13N  done 






ID24Z
5 th ID24Z  done 






ID17S
6 th ID17S  done 






ID7G
7 th ID7G  done 






ID19U
10 th ID19U  done 






ID12M
11 th ID12M  done 






ID11L
12 th ID11L  done 






ID9J
13 th ID9J  done 






ID22X
14 th ID22X  done 






ID8H
15 th ID8H  done 






ID16R
18 th ID16R  done 






ID21W
19 th ID21W  done 






ID15Q
22 th ID15Q  done 






ID23Y
28 th ID23Y  done 






ID1A
36 th ID1A  done 






ID2B
41 th ID2B  done 






ID10K
45 th ID10K  done 






ID5E
52 th ID5E  done 






ID14P
53 th ID14P  done 






ID3C
66 th ID3C  done 






ID6F
87 th ID6F  done 








In [4]:
df_init = pd.DataFrame()

date_str = '_R13'

manual_seg_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/label_bedrest/'
auto_seg_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/seg_bedrest/'
img_dir = 'nnUNet_raw_data_base/nnUNet_train_data_raw/20240808/AFL_TASS_BestRest_Test/img_bedrest/'
csv_write_path= 'csv_and_figure/MRI_Bedrest_fat_ratio_R13_24.csv'

pid_list = []

for idx, d in sorted(enumerate(os.listdir(manual_seg_dir))):
    if '.nii.gz' in d :
#         print(d,'processing')

        indices = [i for i, c in enumerate(d) if c == '_']
        pid = d[:indices[0]]
        if pid not in pid_list:
            pid_list.append(pid)
            print(pid)
            pid_fat = pid + '_R13Fat'
            pid_water = pid + '_R13Water'
            
            mskCube_fat, prdCube_fat, imgCube_fat = \
                get_msk_prd_img_cube_by_id(pid_fat, manual_seg_dir, auto_seg_dir, img_dir )
            
            mskCube_water, prdCube_water, imgCube_water = \
                get_msk_prd_img_cube_by_id(pid_water, manual_seg_dir, auto_seg_dir, img_dir)
            
            mskCube_fat1, mskCube_fat2, mskCube_fat3, mskCube_fat4 = get_cube_by_class(mskCube_fat)
            mskCube_water1, mskCube_water2, mskCube_water3, mskCube_water4 = get_cube_by_class(mskCube_water)            
            prdCube_fat1, prdCube_fat2, prdCube_fat3, prdCube_fat4 = get_cube_by_class(prdCube_fat)  
            prdCube_water1, prdCube_water2, prdCube_water3, prdCube_water4 = get_cube_by_class(prdCube_water)  
            
            
            
            ratio1_gd = get_ratio_of_dixon_image_pair(mskCube_fat1, imgCube_fat, mskCube_water1, imgCube_water)
            ratio2_gd = get_ratio_of_dixon_image_pair(mskCube_fat2, imgCube_fat, mskCube_water2, imgCube_water)
            ratio3_gd = get_ratio_of_dixon_image_pair(mskCube_fat3, imgCube_fat, mskCube_water3, imgCube_water)
            ratio4_gd= get_ratio_of_dixon_image_pair(mskCube_fat4, imgCube_fat, mskCube_water4, imgCube_water)   
            
            ratio1_ai = get_ratio_of_dixon_image_pair(prdCube_fat1, imgCube_fat, prdCube_fat1, imgCube_water)
            ratio2_ai = get_ratio_of_dixon_image_pair(prdCube_fat2, imgCube_fat, prdCube_fat2, imgCube_water)
            ratio3_ai = get_ratio_of_dixon_image_pair(prdCube_fat3, imgCube_fat, prdCube_fat3, imgCube_water)
            ratio4_ai = get_ratio_of_dixon_image_pair(prdCube_fat4, imgCube_fat, prdCube_fat4, imgCube_water) 
            
            df = pd.DataFrame({'MRI_id': pid + '_R13',
                               
                        'ratio1_gd':round(ratio1_gd,5), 'ratio1_ai':round(ratio1_ai,5),
                        'ratio2_gd':round(ratio2_gd,5), 'ratio2_ai':round(ratio2_ai,5),
                        'ratio3_gd':round(ratio3_gd,5), 'ratio3_ai':round(ratio3_ai,5),
                        'ratio4_gd':round(ratio4_gd,5), 'ratio4_ai':round(ratio4_ai,5)
                                                 
                            },index=[0])
            df_init = df_init._append(df)
            df_init.to_csv(csv_write_path )
            print(idx,'th',pid,' done \n\n')
            
            print('\n\n\n')


ID4D
0 th ID4D  done 






ID18T
1 th ID18T  done 






ID20V
2 th ID20V  done 






ID13N
3 th ID13N  done 






ID24Z
5 th ID24Z  done 






ID17S
6 th ID17S  done 






ID7G
7 th ID7G  done 






ID19U
10 th ID19U  done 






ID12M
11 th ID12M  done 






ID11L
12 th ID11L  done 






ID9J
13 th ID9J  done 






ID22X
14 th ID22X  done 






ID8H
15 th ID8H  done 






ID16R
18 th ID16R  done 






ID21W
19 th ID21W  done 






ID15Q
22 th ID15Q  done 






ID23Y
28 th ID23Y  done 






ID1A
36 th ID1A  done 






ID2B
41 th ID2B  done 






ID10K
45 th ID10K  done 






ID5E
52 th ID5E  done 






ID14P
53 th ID14P  done 






ID3C
66 th ID3C  done 






ID6F
87 th ID6F  done 






